<a href="https://colab.research.google.com/github/colincockburn/CISC_473_project/blob/main/image_restoration_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Run this cell at start of each session - Change BRANCH name to your working branch

# ==== Team Bootstrap (Colab) ====
# 1) Mount Drive (data only)
from google.colab import drive
drive.mount('/content/drive')

# 2) Config
REPO_URL  = "https://github.com/colincockburn/CISC_473_project.git"
REPO_DIR  = "/content/drive/MyDrive/projects/CISC_473_project/"
DATA_ROOT = "/content/drive/MyDrive/projects/CISC_473_project_data"

# 3) Clone repo if missing
import os, subprocess, sys
if not os.path.isdir(REPO_DIR):
    os.makedirs(os.path.dirname(REPO_DIR), exist_ok=True)
    print("Cloning repo to", REPO_DIR)
    !git clone "$REPO_URL" "$REPO_DIR"

# 4) Git identity (first time only; uncomment and change to your info)'
# dont push changes

# %cd $REPO_DIR
# !git config user.email "<your_email>@example.com"
# !git config user.name "<Your Name>"

# 5) Sync with origin/main and create/update your feature branch
BRANCH = None  # e.g., feature/cc-dataloader
if BRANCH:
  !git fetch origin
  !git checkout main
  !git pull origin main
  !git checkout -B "$BRANCH"

# 6) Sanity checks
import os
assert os.path.isdir(DATA_ROOT), f"Missing data at: {DATA_ROOT}"
print("OK: repo at", REPO_DIR)
print("OK: data at", DATA_ROOT)
!git status


Mounted at /content/drive


In [ ]:
#load config

In [ ]:
# run training

In [ ]:
# monitor metrics

In [ ]:
#eval model

In [ ]:
# disply results